# AD9833 Modulators test with FTDI
https://www.analog.com/media/en/technical-documentation/data-sheets/ad9833.pdf

In [4]:
import os, sys

def gen_relative_path(target_path): 
    
    def del_same_parents(target_path, current_path):
        if len(target_path) * len(current_path) > 0:
            if target_path[0] == current_path[0]:
                target_path.pop(0)
                current_path.pop(0)
                del_same_parents(target_path, current_path)
            
    current_path = os.getcwd().replace('\\', '/').split('/')
    target_path = target_path.replace('\\', '/').split('/')
    del_same_parents(target_path, current_path)
    
    return ['..'] * len(current_path) + target_path


def append_source_relative_path(source_relative_path):
    sys.path.append(os.sep.join(source_relative_path))

In [5]:
# paths = ['']
# paths = [gen_relative_path(p) for p in paths]
# print(paths)

In [6]:
paths = [['..', '..', '..', '..', 'ORM', 'bitbucket', 'github', 'codes'],
         ['..', '..', '..', '..', 'USB', 'Universal Serial Bus', 'bitbucket', 'github', 'codes'], 
         ['..', '..', '..', '..', '..', '已完成', 'Bridges', 'bitbucket', 'github', 'codes'], 
         ['..', 'codes']]

for path in paths:
    append_source_relative_path(path)

In [7]:
%pylab inline 

from signal_generators import adapters
from signal_generators.ad983x.ad9833 import AD9833
from signal_generators.ad983x.ad9833 import modulators
from signal_generators.ad983x.ad9833 import tools
from signal_generators.shift_register import ShiftRegister

Populating the interactive namespace from numpy and matplotlib
No USB device matches URL ftdi://ftdi:ft232h/1


## Debug mode?

In [8]:
adapters.SPI.DEBUG_MODE = False  # whether to show SPI written data.
AD9833.DEBUG_MODE = False        # whether to dump registers. 

## Generators and Modulators

In [9]:
with_hardware_device = False

if with_hardware_device:
    _clk = adapters.Pin.get_Ftdi_pin(pin_id = 4)
    _data = adapters.Pin.get_Ftdi_pin(pin_id = 1)

    _ss1 = adapters.Pin.get_Ftdi_pin(pin_id = 3)
    _ss2 = adapters.Pin.get_Ftdi_pin(pin_id = 0)

    _spi_1 = ShiftRegister(stb_pin = _ss1, clk_pin = _clk, data_pin = _data, polarity = 1)
    _spi_2 = ShiftRegister(stb_pin = _ss2, clk_pin = _clk, data_pin = _data, polarity = 1)
else:
    _spi_1 = _spi_2 = _ss1 = _ss2 = None  # using None for testing without actual hardware device.


ad1 = AD9833(_spi_1, _ss1)
ad2 = AD9833(_spi_2, _ss2)

In [10]:
_spi = _spi_1

bfsk = modulators.BFSK(_spi_1, _ss1)
bpsk = modulators.BPSK(_spi_1, _ss1)
dtmf = modulators.DTMF(_spi_1, _ss1, _ss2)
fm = modulators.FM(_spi_1, _ss1)
iq = modulators.IQ(_spi_1, _ss1, _ss2)
ook = modulators.OOK(_spi_1, _ss1)
pm = modulators.PM(_spi_1, _ss1)
pwm = modulators.PWM(_spi_1, _ss1)
qpsk = modulators.QPSK(_spi_1, _ss1)
 
ad1.reset()
ad2.reset()

## Testing data sequence

In [11]:
samples_size = 100
duration = 0.1

symbols = np.random.randint(2, size = samples_size)
digital_sequence = [(symbol, duration) for symbol in symbols] 


symbols = np.random.uniform(-1, 1, size = samples_size).round(5)
analog_sequence = [(symbol, duration) for symbol in symbols] 


symbols = np.random.choice(['1', '2', '3', 'A', '4', '5', '6', 'B', '7', '8', '9', 'C', '*', '0', '#', 'D'], 
                           size = samples_size)
dtmf_sequence = [(symbol, duration) for symbol in symbols] 


symbols = np.random.choice([0, 1, 2, 3], size = samples_size)
quadrature_sequence = [(symbol, duration) for symbol in symbols] 

## Modulators test

In [ ]:
bfsk.send_sequence(digital_sequence)

In [ ]:
bpsk.send_sequence(digital_sequence)

In [ ]:
ook.send_sequence(digital_sequence)

In [ ]:
fm.send_sequence(analog_sequence)

In [ ]:
pm.send_sequence(analog_sequence)

In [ ]:
qpsk.send_sequence(quadrature_sequence)

In [ ]:
iq.send_sequence(quadrature_sequence)

In [ ]:
dtmf.send_sequence(dtmf_sequence)

In [ ]:
pwm.run()